# **Code Principale**

In [ ]:
import h5py
import numpy as np
from matplotlib.pyplot import*
import time
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import svm
from sklearn.metrics import accuracy_score
import math
import pandas as pd
from sklearn.preprocessing import StandardScaler
from collections import Counter

In [ ]:
def kernel(x=1,y=2,segma=0.1):
    val=np.exp(-np.linalg.norm(x-y)**2/(2*segma**2)); #%Gaussian kernel
    return val

In [ ]:
def f_model(x,w,b=1):
    val=w.T@x+b;
    return val

def model(x,w,b=1):
    m,l=x.shape
    y=np.zeros((l,1))
    for i in range(l):
        y[i]=f_model(x[:,i],w,b);
    return y.T

In [ ]:
def model_c(x0,x,sigma):
    n,m=x.shape
    c,l=x0.shape
    k=np.zeros((c,n))
    for i in range(c):
        for j  in range(n):
            k[i,j]=kernel(x0[i,:],x[j,:],sigma)
    return k

def model_test(K,w,b):
    l,m=K.shape
    y=np.zeros((l,1))
    for i in range(l):
        y[i]=f_model(K[i,:],w,b);
    return y

## Newton

In [ ]:
def df_nweton(y,a,t,mu,n):
    l=len(y);
    val=np.ones((l,1))
    for i in range(l):
        val[i]=1/((mu/n)*(pow((a*y[i]),2))*np.exp(a*y[i]*t[i])+1)
    dF=np.diagflat(val)
    #print(dF.shape)
    return dF

In [ ]:
def F_newton(y,a,t,Z1,Z2,P,mu,C,n):
    l=len(y);
    F=np.zeros((l,1))
    for i in range(l):
      F[i]=t[i]-(P[i]-(mu*C)/2*(Z1[i]+Z2[i]))+(1/n)*(mu*a*y[i])*(np.exp(a*y[i]*t[i])-1)+C*mu;
    return F

In [ ]:
def newtonMethod(y,a,Z1,Z2,P,mu,lam,nmax,n):
    error=1;
    itcount=1;
    l=len(y);
    t_old=np.ones((l,1));
    while (itcount <= nmax):
          x=F_newton(y,a,t_old,Z1,Z2,P,mu,lam,n);
          b=df_nweton(y,a,t_old,mu,n);
          t=t_old-np.dot(b,x)
          #error = np.linalg.norm(t - t_old)
          t_old=t;
          itcount=itcount+1;
    return t_old

### Primal dual algorithm

In [ ]:
def Primal_Dual_C(A,X0,N,y,C,a,mu,tau):
    k=1
    err_r=[]
    acc=[]
    b=1
    ya=np.sign(model(A.T,X0,b));
    acc.append(accuracy_score(ya.T, y))
    err_r.append(1)
    n=len(y);
    Y=np.diag(y);
    W=X0;
    W_old=W;
    Z2=np.zeros((n,1))+0.01
    Z1=np.zeros((n,1))+0.01
    Z2_old=np.zeros((n,1))+0.01
    Z1_old=np.zeros((n,1))+0.01
    X=np.zeros((n,1))+0.01
    T=np.zeros((n,1))+0.01
    P=np.zeros((n,1))+0.01
    P_old=np.zeros((n,1))+0.01
    delta=0.1
    om=1;
    y=y.reshape((n,1))
    t=np.ones((n,1));
    nu=0.9
    while (k <= N):
          #Calcule du gradient%%%%%%%
          W=(W_old-(1/2)*mu*C*((Y@A).T@Z1_old-A.T@y))/(1+mu);
          #Calcule de Phi %%%%%%%
          P=newtonMethod(y,a,Z1_old,Z2_old,P_old,mu,C,5,1)
          X=W+om*(W-W_old);
          T=P+om*(P-P_old);
          R=Z1_old+(delta*C/2)*((Y@A)@X+T);
          Z1=R/max(np.linalg.norm(R,np.inf),1);
          B=Z2_old+(delta*C/2)*T;
          Z2=B/max(np.linalg.norm(B,np.inf),1);
          err_r.append(np.linalg.norm(W_old- W)/np.linalg.norm(W_old))
          ya=np.sign(model(A.T,W,b));
          acc.append(accuracy_score(ya.T, y))
          W_old= W;
          Z1_old=Z1;
          Z2_old=Z2;
          P_old=P;
          k=k+1;

          """om=1/np.sqrt(1+2*tau*mu);
          mu=mu*om;
          nu=nu/om;"""
          if(k%20==0):
              print(k,'iteration ===> ACC= ',acc[k-1],' and ERR=',err_r[k-1])
    return W,ya,err_r


In [ ]:
def train_model(X,y,C,a,n):
    #n=1000;
    #C=1e-1;
    e,f=X.shape
    #X0=np.random.randn(f,1)
    X0=np.zeros((f,1))+0.1;
    W,ya,err=Primal_Dual_C(X,X0,n,y,C,a,0.1,0.0005);
    return W,err

# **haberman**

In [ ]:
from google.colab import files
uploaded = files.upload()
import io
df2 = pd.read_csv(io.BytesIO(uploaded['haberman.csv']))
# Dataset is now stored in a Pandas Dataframe

Saving haberman.csv to haberman.csv


In [ ]:
data = pd.read_csv("haberman.csv")
data.head()

,Age,Op_Year,axil_nodes,Surv_status
0,30,64,1,1
1,30,62,3,1
2,30,65,0,1
3,31,59,2,1
4,31,65,4,1


In [ ]:
def test_cmc(c,a,n):
    data = pd.read_csv("haberman.csv")
    data.head()
    #data = data.dropna()
    #data.isnull().sum()
    #training and test datasets
    X = data[["Age","Op_Year","axil_nodes"]]
    y = data["Surv_status"]
    y=np.array(y)
    scaler = StandardScaler()
    scaler.fit(X)
    X_scaled = scaler.transform(X)
    X_scaled = pd.DataFrame(data=X_scaled, index=X.index, columns= X.columns)
    for i in range(len(y)):
        if y[i] == 2:
            y[i]=-1
    X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=10)#10
    X_train=np.array(X_train);
    Y_train=np.array(Y_train);
    X_test=np.array(X_test);
    Y_test=np.array(Y_test);
    Xmin,X2=train_model(X_train,Y_train,c,a,n)
    y_p=np.sign(model(X_test.T,Xmin,1));
    #Y_test=Y_test.reshape((len(Y_test),1))

    acc=accuracy_score(y_p.T,Y_test)
    return y_p,acc

In [ ]:
#X_num,X_cat,y=load_data()
y_p,acc=test_cmc(1e1,-1,100)
print(acc)




20 iteration ===> ACC=  0.7172131147540983  and ERR= 0.016982450689105918
40 iteration ===> ACC=  0.7172131147540983  and ERR= 0.0019704355383725358
60 iteration ===> ACC=  0.7172131147540983  and ERR= 8.957153885250344e-05
80 iteration ===> ACC=  0.7172131147540983  and ERR= 0.00024200422321258222
100 iteration ===> ACC=  0.7172131147540983  and ERR= 0.0003016106990926821
0.8064516129032258


# **liver_disorder**

In [ ]:
from google.colab import files
uploaded = files.upload()
import io
df2 = pd.read_csv(io.BytesIO(uploaded['liver_disorder.csv']))
# Dataset is now stored in a Pandas Dataframe


Saving liver_disorder.csv to liver_disorder.csv


In [ ]:
def test_l(c,a,n,r=7):
    data = pd.read_csv("liver_disorder.csv")
    data.head()
    #data = data.dropna()
    #data.isnull().sum()
    #training and test datasets
    X = data[["mcv","alkphos","sgpt","sgot","gammagt","drinks"]]
    y = data["selector"]
    y=np.array(y)
    scaler = StandardScaler()
    scaler.fit(X)
    X_scaled = scaler.transform(X)
    X_scaled = pd.DataFrame(data=X_scaled, index=X.index, columns= X.columns)
    for i in range(len(y)):
        if y[i] == 0:
            y[i]=-1
    X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=r)
    X_train=np.array(X_train);


    Y_train=np.array(Y_train);
    X_test=np.array(X_test);
    Y_test=np.array(Y_test);
    Xmin,X2=train_model(X_train,Y_train,c,a,n)
    y_p=np.sign(model(X_test.T,Xmin,1));
    #Y_test=Y_test.reshape((len(Y_test),1))
    from sklearn import metrics
    print(metrics.classification_report(Y_test, y_p.T))
    acc=accuracy_score(y_p.T,Y_test)
    return y_p,acc

In [ ]:
y_p,acc=test_l(1,1,100,91)
print(acc)





20 iteration ===> ACC=  0.6360294117647058  and ERR= 0.01748349655063808
40 iteration ===> ACC=  0.6360294117647058  and ERR= 0.0021285864378294936
60 iteration ===> ACC=  0.6360294117647058  and ERR= 0.00027001033207297545
80 iteration ===> ACC=  0.6360294117647058  and ERR= 2.1650256141521473e-05
100 iteration ===> ACC=  0.6360294117647058  and ERR= 4.1206451978823e-05
              precision    recall  f1-score   support

          -1       0.71      0.82      0.76        33
           1       0.81      0.69      0.75        36

    accuracy                           0.75        69
   macro avg       0.76      0.76      0.75        69
weighted avg       0.76      0.75      0.75        69

0.7536231884057971


# **Ionosphere**

In [ ]:
from google.colab import files
uploaded = files.upload()
import io
df2 = pd.read_csv(io.BytesIO(uploaded['Ionosphere.csv']))
# Dataset is now stored in a Pandas Dataframe


Saving Ionosphere.csv to Ionosphere.csv


In [ ]:
def test_I(c,a,n,r=7):
    data = pd.read_csv("Ionosphere.csv")
    data.head()
    #data = data.dropna()
    #data.isnull().sum()
    #training and test datasets
    X = data[["V1","V2","V3","V4","V5","V6",
              "V7","V8","V9","V10","V11",
              "V12","V13","V14","V15","V16",
              "V17","V18","V19","V20","V21",
              "V22","V23","V24","V25","V26",
              "V27","V28","V29","V30","V31",
              "V32","V33","V34"]]
    y = data["Class"]
    y=np.array(y)
    scaler = StandardScaler()
    scaler.fit(X)
    X_scaled = scaler.transform(X)
    X_scaled = pd.DataFrame(data=X_scaled, index=X.index, columns= X.columns)
    for i in range(len(y)):
        if y[i] == 0:
            y[i]=-1
    X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=r)
    X_train=np.array(X_train);
    Y_train=np.array(Y_train);
    X_test=np.array(X_test);
    Y_test=np.array(Y_test);
    Xmin,X2=train_model(X_train,Y_train,c,a,n)
    y_p=np.sign(model(X_test.T,Xmin,1));
    #Y_test=Y_test.reshape((len(Y_test),1))
    from sklearn import metrics
    print(metrics.classification_report(Y_test, y_p.T))
    acc=accuracy_score(y_p.T,Y_test)
    return y_p,acc

In [ ]:
y_p,acc=test_I(0.01,0.1,100,3)
print(acc)





20 iteration ===> ACC=  0.8178571428571428  and ERR= 0.013250645132826996
40 iteration ===> ACC=  0.8357142857142857  and ERR= 0.002336575553259017
60 iteration ===> ACC=  0.8464285714285714  and ERR= 0.0032498779392200546
80 iteration ===> ACC=  0.8571428571428571  and ERR= 0.0033171406630167527
100 iteration ===> ACC=  0.8607142857142858  and ERR= 0.003175738186971224
              precision    recall  f1-score   support

          -1       0.95      0.83      0.88        23
           1       0.92      0.98      0.95        48

    accuracy                           0.93        71
   macro avg       0.94      0.90      0.92        71
weighted avg       0.93      0.93      0.93        71

0.9295774647887324


# **australian**

In [ ]:
from google.colab import files
uploaded = files.upload()
import io
df2 = pd.read_csv(io.BytesIO(uploaded['australian.csv']))
# Dataset is now stored in a Pandas Dataframe


Saving australian.csv to australian.csv


In [ ]:
data = pd.read_csv("australian.csv")
data.head()


,CustomerID,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,Class
0,15776156,1,22.08,11.46,2,4,4,1.585,0,0,0,1,2,100,1213,0
1,15739548,0,22.67,7.00,2,8,4,0.165,0,0,0,0,2,160,1,0
2,15662854,0,29.58,1.75,1,4,4,1.250,0,0,0,1,2,280,1,0
3,15687688,0,21.67,11.50,1,5,3,0.000,1,1,11,1,2,0,1,1
4,15715750,1,20.17,8.17,2,6,4,1.960,1,1,14,0,2,60,159,1


In [ ]:
def test_au(c,a,n,r=7):
    data = pd.read_csv("australian.csv")
    data.head()
    X = data[["A1","A2","A3","A4","A5","A6","A7","A8","A9","A10","A11","A12","A13","A14"]]
    y = data['Class']
    y=np.array(y)
    scaler = StandardScaler()
    scaler.fit(X)
    X_scaled = scaler.transform(X)
    X_scaled = pd.DataFrame(data=X_scaled, index=X.index, columns= X.columns)
    for i in range(len(y)):
        if y[i] == 0:
            y[i]=-1
    X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=r)
    X_train=np.array(X_train);
    Y_train=np.array(Y_train);
    X_test=np.array(X_test);
    Y_test=np.array(Y_test);
    Xmin,X2=train_model(X_train,Y_train,c,a,n)
    y_p=np.sign(model(X_test.T,Xmin,1));
    #Y_test=Y_test.reshape((len(Y_test),1))
    from sklearn import metrics
    print(metrics.classification_report(Y_test, y_p.T))
    acc=accuracy_score(y_p.T,Y_test)
    return y_p,acc

In [ ]:
y_p,acc=test_au(1,1,100,10)
print(acc)





20 iteration ===> ACC=  0.8586956521739131  and ERR= 0.018189997583204196
40 iteration ===> ACC=  0.8586956521739131  and ERR= 0.0022814955990652563
60 iteration ===> ACC=  0.8586956521739131  and ERR= 0.00032775701278612863
80 iteration ===> ACC=  0.8586956521739131  and ERR= 4.7002741452441404e-05
100 iteration ===> ACC=  0.8586956521739131  and ERR= 5.7854908781289846e-06
              precision    recall  f1-score   support

          -1       0.96      0.88      0.92        80
           1       0.85      0.95      0.89        58

    accuracy                           0.91       138
   macro avg       0.90      0.91      0.90       138
weighted avg       0.91      0.91      0.91       138

0.9057971014492754


# **diabetes**

In [ ]:
from google.colab import files
uploaded = files.upload()
import io
df2 = pd.read_csv(io.BytesIO(uploaded['diabetes-dataset.csv']))
# Dataset is now stored in a Pandas Dataframe


Saving diabetes-dataset.csv to diabetes-dataset.csv


In [ ]:
def test_d(c,a,n,r=7):
    data = pd.read_csv("diabetes-dataset.csv")
    data.head()
    X = data[["Pregnancies",	"Glucose",	"BloodPressure",	"SkinThickness",	"Insulin",	"BMI",	"DiabetesPedigreeFunction",	"Age"]]
    X=X[0:768]
    y = data['Outcome']
    y=y[0:768]
    y=np.array(y)
    scaler = StandardScaler()
    scaler.fit(X)
    X_scaled = scaler.transform(X)
    X_scaled = pd.DataFrame(data=X_scaled, index=X.index, columns= X.columns)
    for i in range(len(y)):
        if y[i] == 0:
            y[i]=-1
    X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=r)
    X_train=np.array(X_train);


    Y_train=np.array(Y_train);
    X_test=np.array(X_test);
    Y_test=np.array(Y_test);
    Xmin,X2=train_model(X_train,Y_train,c,a,n)
    y_p=np.sign(model(X_test.T,Xmin,1));
    #Y_test=Y_test.reshape((len(Y_test),1))
    from sklearn import metrics
    print(metrics.classification_report(Y_test, y_p.T))
    acc=accuracy_score(y_p.T,Y_test)
    return y_p,acc

In [ ]:
y_p,acc=test_d(2.1,0.5,100,5)
print(acc)





20 iteration ===> ACC=  0.7263843648208469  and ERR= 0.016436252039744675
40 iteration ===> ACC=  0.7263843648208469  and ERR= 0.00233719568336462
60 iteration ===> ACC=  0.7263843648208469  and ERR= 0.000564625297742441
80 iteration ===> ACC=  0.7247557003257329  and ERR= 0.00030835980357055446
100 iteration ===> ACC=  0.7247557003257329  and ERR= 0.0002676409179515769
              precision    recall  f1-score   support

          -1       0.89      0.74      0.81        98
           1       0.65      0.84      0.73        56

    accuracy                           0.78       154
   macro avg       0.77      0.79      0.77       154
weighted avg       0.80      0.78      0.78       154

0.7792207792207793


# **sonar**

In [ ]:
from google.colab import files
uploaded = files.upload()
import io
df2 = pd.read_csv(io.BytesIO(uploaded['sonar.csv']))
# Dataset is now stored in a Pandas Dataframe


Saving sonar.csv to sonar.csv


In [ ]:
data = pd.read_csv("sonar.csv",names=range(0, 61),header=None)
data.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


In [ ]:
def test_s(c,a,n,r=7):
    data = pd.read_csv("sonar.csv",names=range(0, 61),header=None)
    data.head()
    names = [0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
            51, 52, 53, 54, 55, 56, 57, 58, 59, 60]
    data.columns = names
    data['Class']=np.where(data[60]==data[60][1],1,-1)
    X = data[[0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
            51, 52, 53, 54, 55, 56, 57, 58, 59]]
    y = data['Class']
    scaler = StandardScaler()
    scaler.fit(X)
    X_scaled = scaler.transform(X)
    X_scaled = pd.DataFrame(data=X_scaled, index=X.index, columns= X.columns)
    y=np.array(y);
    X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=r)
    X_train=np.array(X_train);
    Y_train=np.array(Y_train);
    X_test=np.array(X_test);
    Y_test=np.array(Y_test);
    Xmin,X2=train_model(X_train,Y_train,c,a,n)
    y_p=np.sign(model(X_test.T,Xmin,1));
    #Y_test=Y_test.reshape((len(Y_test),1))
    from sklearn import metrics
    print(metrics.classification_report(Y_test, y_p.T))
    acc=accuracy_score(y_p.T,Y_test)
    return y_p,acc

In [ ]:
y_p,acc=test_s(1,1,100,99)
print(acc)





20 iteration ===> ACC=  0.7710843373493976  and ERR= 0.017658656085038135
40 iteration ===> ACC=  0.7710843373493976  and ERR= 0.0022470231476250002
60 iteration ===> ACC=  0.7710843373493976  and ERR= 0.0003326388877004007
80 iteration ===> ACC=  0.7710843373493976  and ERR= 5.38265542383479e-05
100 iteration ===> ACC=  0.7710843373493976  and ERR= 1.1163762600235576e-05
              precision    recall  f1-score   support

          -1       0.84      0.84      0.84        19
           1       0.87      0.87      0.87        23

    accuracy                           0.86        42
   macro avg       0.86      0.86      0.86        42
weighted avg       0.86      0.86      0.86        42

0.8571428571428571


# **wpbc**

In [ ]:
from google.colab import files
uploaded = files.upload()
import io
df2 = pd.read_csv(io.BytesIO(uploaded['wpbc.csv']))
# Dataset is now stored in a Pandas Dataframe


Saving wpbc.csv to wpbc.csv


In [ ]:
data = pd.read_csv("wpbc.csv")
data.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


In [ ]:
def test_w(c,a,n,r=7):
    data = pd.read_csv("wpbc.csv")
    data.drop(['id'], axis = 1, inplace = True)
    data.drop(['Unnamed: 32'], axis = 1, inplace = True)
    data['diagnosis'] = data['diagnosis'].map({'M': -1, 'B': 1})
    data.head()
    #data = data.dropna()
    #data.isnull().sum()
    #training and test datasets
    X = data[["radius_mean","texture_mean",
              "perimeter_mean","area_mean","smoothness_mean",
              "compactness_mean","concavity_mean",
              "concave points_mean","symmetry_mean",
              "fractal_dimension_mean","radius_se","texture_se",
              "perimeter_se","area_se","smoothness_se","compactness_se",
              "concavity_se","concave points_se","symmetry_se","fractal_dimension_se",
              "radius_worst","texture_worst","perimeter_worst",
              "area_worst","smoothness_worst","compactness_worst",
              "concavity_worst","concave points_worst","symmetry_worst","fractal_dimension_worst"]]
    y = data["diagnosis"]
    y=np.array(y)
    scaler = StandardScaler()
    scaler.fit(X)
    X_scaled = scaler.transform(X)
    X_scaled = pd.DataFrame(data=X_scaled, index=X.index, columns= X.columns)
    for i in range(len(y)):
        if y[i] == 'B':
            y[i]=-1
        if y[i] == 'M':
            y[i]=1
    X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=r)
    X_train=np.array(X_train);


    Y_train=np.array(Y_train);
    X_test=np.array(X_test);
    Y_test=np.array(Y_test);
    Xmin,X2=train_model(X_train,Y_train,c,a,n)
    y_p=np.sign(model(X_test.T,Xmin,1));
    #Y_test=Y_test.reshape((len(Y_test),1))
    from sklearn import metrics
    print(metrics.classification_report(Y_test, y_p.T))
    acc=accuracy_score(y_p.T,Y_test)
    return y_p,acc

In [ ]:
y_p,acc=test_w(1,1,100,99)
print(acc)





20 iteration ===> ACC=  0.9428571428571428  and ERR= 0.019037172350425662
40 iteration ===> ACC=  0.9428571428571428  and ERR= 0.002399035152867708
60 iteration ===> ACC=  0.9428571428571428  and ERR= 0.0003484462068036018
80 iteration ===> ACC=  0.9428571428571428  and ERR= 5.1591308119802934e-05
100 iteration ===> ACC=  0.9428571428571428  and ERR= 7.663615123543935e-06
              precision    recall  f1-score   support

          -1       0.89      0.84      0.86        38
           1       0.92      0.95      0.94        76

    accuracy                           0.91       114
   macro avg       0.91      0.89      0.90       114
weighted avg       0.91      0.91      0.91       114

0.9122807017543859


# **German**

In [ ]:
from google.colab import files
uploaded = files.upload()
import io
df2 = pd.read_csv(io.BytesIO(uploaded['SouthGermanCredit.asc']))
# Dataset is now stored in a Pandas Dataframe


In [ ]:
def test_g(c,a,n,r=7):
    data= pd.read_csv('SouthGermanCredit.asc',sep=' ')
    columns =['status','duration','credit_history','purpose','amount','savings','employment_duration',
          'installment_rate','personal_status_sex','other_debtors','present_residence','property',
          'age','other_installment_plans','housing','number_credits','job','people_liable','telephone',
          'foreign_worker','credit_risk']
    data.columns = columns
    X=data[['status','duration','credit_history','purpose','amount','savings','employment_duration',
          'installment_rate','personal_status_sex','other_debtors','present_residence','property',
          'age','other_installment_plans','housing','number_credits','job','people_liable','telephone',
          'foreign_worker']]
    y=data['credit_risk']
    y=np.array(y)
    scaler = StandardScaler()
    scaler.fit(X)
    X_scaled = scaler.transform(X)
    X_scaled = pd.DataFrame(data=X_scaled, index=X.index, columns= X.columns)

    for i in range(len(y)):
        if y[i] == 0:
           y[i]=-1
    X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=r)
    X_train=np.array(X_train);
    #X_train= X_train+0.05*(2*np.random.random(X_train.shape)-1);
    Y_train=np.array(Y_train);
    X_test=np.array(X_test);
    Y_test=np.array(Y_test);
    Xmin,X2=train_model(X_train,Y_train,c,a,n)
    y_p=np.sign(model(X_test.T,Xmin,1));
    #Y_test=Y_test.reshape((len(Y_test),1))
    from sklearn import metrics
    print(metrics.classification_report(Y_test, y_p.T))
    acc=accuracy_score(y_p.T,Y_test)
    return y_p,acc

In [ ]:
#y_p,acc=test_g(0.041,1,100,42)
y_p,acc=test_g(0.01,1,100,7)
print(acc)





# **vote**

In [ ]:
from google.colab import files
uploaded = files.upload()
import io
df2 = pd.read_csv(io.BytesIO(uploaded['house-votes-84-data.csv']))
# Dataset is now stored in a Pandas Dataframe


Saving house-votes-84-data.csv to house-votes-84-data.csv


In [ ]:
def test_v(c,a,n,r=7):
    df= pd.read_csv('house-votes-84-data.csv')
    def replace(column_name):
        mode = df[column_name].mode(dropna=True)[0]
        df[column_name] = df[column_name].replace('?', mode)
        df[column_name] = df[column_name].replace('y', 1)
        df[column_name] = df[column_name].replace('n', 0)
        df[column_name] = df[column_name].replace('republican', 1)
        df[column_name] = df[column_name].replace('democrat', -1)
    f = df.columns
    for column_name in f:
        replace(column_name)

    y = df['party']
    X = df.drop('party', axis = 1)
    X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=r)
    X_train=np.array(X_train);
    Y_train=np.array(Y_train);
    X_test=np.array(X_test);
    Y_test=np.array(Y_test);
    Xmin,X2=train_model(X_train,Y_train,c,a,n)
    y_p=np.sign(model(X_test.T,Xmin,1));
    #Y_test=Y_test.reshape((len(Y_test),1))
    from sklearn import metrics
    print(metrics.classification_report(Y_test, y_p.T))
    acc=accuracy_score(y_p.T,Y_test)
    return y_p,acc

In [ ]:
#y_p,acc=test_g(5,2,100,6)(1,1,100,4)
y_p,acc=test_v(2,0.1,100,21)
print(acc)




20 iteration ===> ACC=  0.9051724137931034  and ERR= 0.04634910790967804
40 iteration ===> ACC=  0.9022988505747126  and ERR= 0.0021326840174654416
60 iteration ===> ACC=  0.9022988505747126  and ERR= 0.0006676676606276247
80 iteration ===> ACC=  0.9022988505747126  and ERR= 0.0005720974633999173
100 iteration ===> ACC=  0.9022988505747126  and ERR= 0.0005544989027630303
              precision    recall  f1-score   support

          -1       0.96      1.00      0.98        51
           1       1.00      0.94      0.97        36

    accuracy                           0.98        87
   macro avg       0.98      0.97      0.98        87
weighted avg       0.98      0.98      0.98        87

0.9770114942528736


# **USPS**

In [ ]:
from google.colab import files
uploaded = files.upload()
##import io
##df2 = pd.read_csv(io.BytesIO(uploaded['usps.h5']))
# Dataset is now stored in a Pandas Dataframe

In [ ]:
import numpy as np

import matplotlib.pyplot as plt

with h5py.File("usps.h5", 'r') as hf:
        train = hf.get('train')
        X_tr = train.get('data')[:]
        y_tr = train.get('target')[:]
        test = hf.get('test')
        X_te = test.get('data')[:]
        y_te = test.get('target')[:]

In [ ]:
indexc =np.where(y_tr == 0)[0]
indexnc=np.where(y_tr != 0)[0]
index=np.concatenate((indexc,indexnc[0:1194]))
np.random.shuffle(index)
print(index.shape)
X=X_tr[indexc,:]
y=y_tr[indexc]
print(y.shape)

(2388,)
(1194,)


In [ ]:
def preprocessing(clas):
  with h5py.File("usps.h5", 'r') as hf:
        train = hf.get('train')
        X_tr = train.get('data')[:]
        y_tr = train.get('target')[:]
        test = hf.get('test')
        X_te = test.get('data')[:]
        y_te = test.get('target')[:]
  indexc =np.where(y_tr == clas)[0]
  indexnc=np.where(y_tr != clas)[0]
  l=len(indexc)
  index=np.concatenate((indexc,indexnc[0:l]))
  np.random.shuffle(index)
  print(index.shape)
  X=X_tr[index,:]
  y=y_tr[index]
  for i in range(len(y)):
        if y[i]!=clas:
           y[i]=-1
        else:
          y[i]=1
  """img=X_tr0[1].reshape(16,16)
  print(y_tr0[1])
  plt.imshow(img,cmap='Greys')
  plt.show()"""
  return X,y

In [ ]:
def Sort_Tuple(tup):
    return(sorted(tup, key = lambda x: x[0]))
def knn(k,X_tr,X_te,m):
    l=[]
    lsort=[]
    pred=[]
    for i in range(m,m+1):
        for j in range(0,l_tr):
            x=np.sqrt(np.sum((np.square(X_te[i,:]-X_tr[j,:])),axis=0))
            l.append((x,y_tr[j]))
        lsort=Sort_Tuple(l)
        for i in range(0,k):
            pred.append(lsort[i][1])
        counter=Counter(pred)
        print(counter.most_common(1)[0][0],y_te[m])
    img=X_te[m].reshape(16,16)
    plt.imshow(img,cmap='Greys')
    plt.show()
knn(4,X_tr,X_te,8)

NameError: ignored

In [ ]:
def test_v(X,y,c,a,n,r=7):
    X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=r)
    X_train=np.array(X_train);
    Y_train=np.array(Y_train);
    X_test=np.array(X_test);
    Y_test=np.array(Y_test);
    Xmin,X2=train_model(X_train,Y_train,c,a,n)
    y_p=np.sign(model(X_test.T,Xmin,1));
    #Y_test=Y_test.reshape((len(Y_test),1))
    from sklearn import metrics
    print(metrics.classification_report(Y_test, y_p.T))
    acc=accuracy_score(y_p.T,Y_test)
    return y_p,acc

In [ ]:
def save_results(test):
    import pickle
    d={}
    d['X_train']=XX
    d['y_train']=yy
    d['acc']=acc
    d['a']=a
    d['c']=c
    d['r']=r
    filename = "%s.dat" % test
    with open(filename, 'wb') as fichier:
    #with open('donnees.dat', 'wb') as fichier:
        mon_pickler = pickle.Pickler(fichier)
        mon_pickler.dump(d)
    files.download(filename)


In [ ]:
def read_results(filename):
    with open(filename, 'rb') as fichier:
    #with open('donnees.dat', 'rb') as fichier:
        mon_depickler = pickle.Unpickler(fichier)
        d= mon_depickler.load()
    data0 = {'X_train':d['X_train'], 'y_train':d['y_train'],'Acc':d['acc'], 'C':d['c'], 'a':d['a'], 'r':d['r']}
    yy=data0['y_train']
    XX=data0['X_train']
    print(data0['Acc'])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
filename = "6.dat"
with open(filename, 'rb') as fichier:
#with open('donnees.dat', 'rb') as fichier:
    mon_depickler = pickle.Unpickler(fichier)
    d= mon_depickler.load()
data0 = {'X_train':d['X_train'], 'y_train':d['y_train']}
yy=data0['y_train']
XX=data0['X_train']

In [ ]:
train_for=9
XX,yy=preprocessing(train_for)

(1288,)


In [ ]:
#0--->y_p,acc=test_v(XX,yy,0.01,1,100,51)
#1--->y_p,acc=test_v(XX,yy,0.01,1,100,7)
#2--->y_p,acc=test_v(XX,yy,0.01,1,100,7)
a=1
c=0.1
n=100
r=51
y_p,acc=test_v(XX,yy,c,a,n,r)
print(acc)

20 iteration ===> ACC=  0.916504854368932  and ERR= 0.015512741437867041
40 iteration ===> ACC=  0.9174757281553398  and ERR= 0.002024357368375613
60 iteration ===> ACC=  0.9174757281553398  and ERR= 0.00029610414206587886
80 iteration ===> ACC=  0.9174757281553398  and ERR= 4.38525537589581e-05
100 iteration ===> ACC=  0.9174757281553398  and ERR= 6.3870547225498126e-06
              precision    recall  f1-score   support

          -1       0.97      0.84      0.90       131
           1       0.86      0.98      0.91       127

    accuracy                           0.91       258
   macro avg       0.91      0.91      0.91       258
weighted avg       0.92      0.91      0.91       258

0.9069767441860465


In [ ]:
save_results(train_for)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
uploaded = files.upload()
filename = "8.dat"
read_results(filename)

Saving 8.dat to 8.dat
0.9124423963133641


In [ ]:
for i in range(10):
   filename = "%s.dat" % i
   read_results(filename)


0.9372384937238494
0.9800995024875622
0.9385665529010239
0.9659090909090909
0.9386973180076629
0.9417040358744395
0.9624060150375939
0.9457364341085271
0.9124423963133641
0.9457364341085271
